In [4]:
import gc
import pickle
import numpy as np
import torch

def datato3d(arrays):#list of np arrays, NULL*3*100
    output=list()
    for i in arrays:
        i=np.transpose(i,(0,2,1))
        output.append(i)
    return output


"""v flip and shuffle"""
def udflip(X_nparray,y_nparray,shuffle=True):
    output=np.zeros((X_nparray.shape[0],X_nparray.shape[1],X_nparray.shape[2]),dtype=np.float32)
    for i in range(X_nparray.shape[0]):
        output[i,:]=np.flipud(X_nparray[i,:])
    output=np.vstack((X_nparray,output))
    y=np.hstack((y_nparray,y_nparray))
    if shuffle:
        shuffle_inx=np.random.permutation(output.shape[0])
        return output[shuffle_inx],y[shuffle_inx]
    else:
        return output,y

with open('../preprocessing/data_61_26.pkl','rb') as f:
    data=pickle.load(f)


dataList=datato3d([data['X_train'],data['X_test']])



In [5]:
data['X_train'].shape, data['X_test'].shape,data['X_train'][4].shape

((3482294, 100, 3), (1511807, 100, 3), (100, 3))

In [6]:
data['X_train'][4]

array([[-0.534638  , -0.13460657, -0.10212614],
       [-0.5315125 , -0.13621436, -0.10441413],
       [-0.5282677 , -0.13790922, -0.10678139],
       [-0.5247717 , -0.13967046, -0.1093182 ],
       [-0.5214086 , -0.14166418, -0.11178002],
       [-0.5186105 , -0.14416443, -0.11404308],
       [-0.51584524, -0.14714974, -0.11625613],
       [-0.51310384, -0.150009  , -0.1176346 ],
       [-0.50995976, -0.15336724, -0.1183257 ],
       [-0.5066227 , -0.15600139, -0.11892449],
       [-0.50256747, -0.15757412, -0.11979838],
       [-0.4981643 , -0.15875036, -0.12061948],
       [-0.49403664, -0.15977065, -0.12102875],
       [-0.4895038 , -0.16071746, -0.1211887 ],
       [-0.48538697, -0.16178721, -0.12117624],
       [-0.48106593, -0.16327563, -0.12096327],
       [-0.47688052, -0.16467458, -0.12064461],
       [-0.4725689 , -0.16565709, -0.1203215 ],
       [-0.4681225 , -0.16666345, -0.11980601],
       [-0.4644146 , -0.16823778, -0.1186564 ],
       [-0.4607663 , -0.17047706, -0.116

In [7]:
X_train=dataList[0]
X_test=dataList[1]
X_train, y_train = X_train[:, :, ::10], data['y_train'][:]
X_test, y_test = X_test[:, :, ::10], data['y_test'][:]

y_test_list=data['y_test'].tolist()

NCLASS=max(y_test_list)+1

X_train,y_train=udflip(X_train,y_train,shuffle=True)
X_test,y_test=udflip(X_test,y_test,shuffle=False)

X_train=torch.from_numpy(X_train)#data['X_train'])
y_train=torch.from_numpy(y_train.astype(np.int32))#data['y_train'])

X_test=torch.from_numpy(X_test)
y_test=torch.from_numpy(y_test.astype(np.int32))  


In [8]:

del data,dataList
gc.collect()
print('data loaded!')
print('X_train_shape',X_train.size())
print('X_test_shape',X_test.size())
print('classes: ', NCLASS)

data loaded!
X_train_shape torch.Size([6964588, 3, 10])
X_test_shape torch.Size([3023614, 3, 10])
classes:  15


In [10]:
X_train.shape

torch.Size([6964588, 3, 10])

In [13]:
import torch.utils.data as utils

kwargs = {}
trn_set=utils.TensorDataset(X_train,y_train)
trn_loader=utils.DataLoader(trn_set,batch_size=2,shuffle=True,**kwargs)

# tst_set=utils.TensorDataset(X_test,y_test)
# tst_loader=utils.DataLoader(tst_set,batch_size=2,shuffle=False,**kwargs)

In [18]:
def compute_tract_anatomical_profile(fiber_roi):
    """
    计算 fiber 轨迹经过的解剖 ROI 统计信息
    """
    unique, counts = np.unique(fiber_roi, return_counts=True)
    roi_profile = dict(zip(unique, counts / len(fiber_roi)))  # 计算占比
    return roi_profile


In [ ]:
for batch_idx,(data,target) in enumerate(trn_loader):
    # data = data[0]
    # target = target[0]
    print(data.shape)
    print(data.shape)
    print(data[:, 3:4, :].shape)
    print(data[:, 3:4, :])
    roi_profile = compute_tract_anatomical_profile(data[:, 3:4, :])
    # print(roi_profile)
    # print(target.shape)
    # print(data)
    # print(target)
    break